In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
import json
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('\nLoad file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    if ds.BodyPartExamined != 'CHEST':
        raise Exception('Incorrect BodyPartExamined')
    if ds.Modality != 'DX':
        raise Exception('Incorrect Modality')
    if ds.PatientPosition not in ['AP', 'PA']:
        raise Exception('Incorrect PatientPosition')
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    
#     idg = ImageDataGenerator(rescale=1. / 255.0)
#     proc_img= idg.flow(img, target_size=img_size)
    img=img/255.0
    proc_img=(img-img_mean)/img_std
    # Resize image to (224,224,3)
#     proc_img = np.zeros((224,224,3))
    proc_img = np.resize(proc_img,(img_size))
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    with open('my_model.json', 'r') as json_file:
        architecture = json.load(json_file)
        model = keras.models.model_from_json(json.dumps(architecture))
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    
    pred = model.predict(img)
    pred = pred[0]
    prediction = 'No pneumonia'
    if pred > thresh:
        prediction = 'Pneumonia'
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'pneumonia_class_2_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 105 # loads the mean image value they used during training preprocessing
img_std = 65 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = .6

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    try:
        img = check_dicom(i)
    except Exception as e:
        print(e)
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)


Load file test1.dcm ...
No pneumonia

Load file test2.dcm ...
No pneumonia

Load file test3.dcm ...
No pneumonia

Load file test4.dcm ...
Incorrect BodyPartExamined

Load file test5.dcm ...
Incorrect Modality

Load file test6.dcm ...
Incorrect PatientPosition
